In [1]:
import pandas as pd
import numpy as np
import json

In [3]:
# Utilizando raw string para la ruta del archivo
archivo_csv = r"C:\Users\JSLV3\Documents\5to Semestre\ETL\Proyecto Water Quality\1ra parte\watersucia.csv"

# Cargar los datos desde el archivo CSV
water = pd.read_csv(archivo_csv, delimiter=';')

# Mostrar las primeras filas del DataFrame
print(water.head())



    Año NombreDepartamento  Div_dpto NombreMunicipio  Divi_muni IrcaMinimo  \
0  2010            Bolívar        13        El Guamo      13248          0   
1  2010            Bolívar        13        El Guamo      13248          0   
2  2010            Bolívar        13        El Guamo      13248          0   
3  2010            Bolívar        13        El Guamo      13248          0   
4  2010            Bolívar        13        El Guamo      13248          0   

  IrcaMaximo IrcaPromedio NombreParametroAnalisis2  MuestrasEvaluadas  \
0        100        37,32        Alcanilidad Total                 67   
1        100        37,32                 Aluminio                 67   
2        100        37,32                 Arsénico                 67   
3        100        37,32                   Cadmio                 67   
4        100        37,32                   Calcio                 67   

   MuestrasTratadas  MuestrasSinTratar  NumeroParametrosMinimo  \
0                67       

## Creando un mayor valor a nuestros datos
### Transformaciones

##### Identificamos los parámetros de análisis que tienen mayor influencia en la contaminación del agua:

In [14]:
parametros_influencia = water.groupby('NombreParametroAnalisis2')['IrcaPromedio'].mean().sort_values(ascending=False)
top_15_parametros = parametros_influencia.head(15)
top_15_parametros

NombreParametroAnalisis2
Giardia                          6.589981
Cryptosporidium                  6.589981
Plaguicidas Totales              6.589981
Trihalometanos totales           6.561441
Molibdeno                        6.561441
Nitratos                         6.560811
Hierro total                     6.560811
Magnesio                         6.560811
Manganeso                        6.560811
Mercurio                         6.560811
Mesófilos                        6.560811
Olor                             6.560811
Nitritos                         6.560811
Fosfatos                         6.560811
Organofosforados y carbamatos    6.560811
Name: IrcaPromedio, dtype: float64

Todos estos parámetros tienen una media de IRCA de aproximadamente 23.94, lo que sugiere una asociación con un nivel de riesgo considerable. Esto puede ser útil para priorizar qué parámetros de calidad del agua necesitan atención más crítica en los programas de monitoreo y tratamiento.

In [ ]:
water = water[water['NombreParametroAnalisis2'].isin(top_15_parametros.index)]

water.head(), water.shape

Hemos filtrado el dataset para dejar solo las filas correspondientes al top 15 parámetros de análisis relacionados con la contaminación del agua y los demas han salido eliminados

##### Clasificación del Índice de Riesgo de la Calidad del Agua (IRCA)

In [6]:
def clasificar_irca(irca):
    try:
        irca = float(irca.replace(',', '.'))
        if irca == 0:
            return 'Sin información'
        elif 0.001 <= irca <= 5:
            return 'Sin riesgo'
        elif 5.001 <= irca <= 14:
            return 'Riesgo bajo'
        elif 14.001 <= irca <= 35:
            return 'Riesgo medio'
        elif 35.001 <= irca <= 80:
            return 'Riesgo alto'
        elif 80.001 <= irca <= 100:
            return 'Inviable sanitariamente'
        else:
            return 'No clasificado'
    except ValueError:
        return 'No clasificado'

water['rango_irca'] = water['IrcaPromedio'].apply(clasificar_irca)

La función clasificar_irca transforma el valor numérico del IrcaPromedio en categorías descriptivas que van desde 'Sin riesgo' hasta 'Inviable sanitariamente', facilitando la interpretación y toma de decisiones en la gestión de la calidad del agua. Esta categorización es crucial para los análisis de salud pública y ambientales, ya que simplifica la visualización de los datos, permite una comparación rápida entre regiones y es esencial para modelar y prever la calidad del agua, lo que resulta en intervenciones más efectivas y políticas basadas en evidencia.

##### Categoría de Tratamiento

In [7]:
def categorize_treatment(row):
    if row['MuestrasTratadas'] == 0:
        return 'Sin tratamiento'
    elif row['MuestrasTratadas'] == row['MuestrasEvaluadas']:
        return 'Tratamiento completo'
    else:
        return 'Tratamiento parcial'

water['TratamientoCategoría'] = water.apply(categorize_treatment, axis=1)
water.head()


,Año,NombreDepartamento,Div_dpto,NombreMunicipio,Divi_muni,IrcaMinimo,IrcaMaximo,IrcaPromedio,NombreParametroAnalisis2,MuestrasEvaluadas,MuestrasTratadas,MuestrasSinTratar,NumeroParametrosMinimo,NumeroParametrosMaximo,NumeroParametrosPromedio,ResultadoMinimo,ResultadoMaximo,ResultadoPromedio,rango_irca,TratamientoCategoría
0,2010,Bolívar,13,El Guamo,13248,0,100,"37,32",Alcanilidad Total,67,67,0,2,7,2,23,23,23,Riesgo alto,Tratamiento completo
1,2010,Bolívar,13,El Guamo,13248,0,100,"37,32",Aluminio,67,67,0,2,7,2,NaN,NaN,NaN,Riesgo alto,Tratamiento completo
2,2010,Bolívar,13,El Guamo,13248,0,100,"37,32",Arsénico,67,67,0,2,7,2,NaN,NaN,NaN,Riesgo alto,Tratamiento completo
3,2010,Bolívar,13,El Guamo,13248,0,100,"37,32",Cadmio,67,67,0,2,7,2,NaN,NaN,NaN,Riesgo alto,Tratamiento completo
4,2010,Bolívar,13,El Guamo,13248,0,100,"37,32",Calcio,67,67,0,2,7,2,14,14,14,Riesgo alto,Tratamiento completo


La columna "Categoría de Tratamiento" clasifica cada conjunto de muestras de agua según el grado de tratamiento que han recibido. Esta clasificación ayuda a entender la gestión y eficacia de los procesos de tratamiento implementados en diferentes localidades. Las categorías son:

•	Sin tratamiento: Indica que ninguna de las muestras evaluadas fue tratada.

•	Tratamiento parcial: Indica que una parte de las muestras evaluadas fue tratada, pero no todas.

•	Tratamiento completo: Indica que todas las muestras evaluadas fueron tratadas.


##### Tasa de tratamiento

In [8]:
water['TasaTratamiento'] = water['MuestrasTratadas'] / water['MuestrasEvaluadas']
water['TasaTratamiento'].replace([float('inf'), float('-inf'), pd.NA], 0, inplace=True)
water[['NombreDepartamento', 'NombreMunicipio', 'Año', 'MuestrasEvaluadas', 'MuestrasTratadas', 'TasaTratamiento']].head()

C:\Users\JSLV3\AppData\Local\Temp\ipykernel_10352\1244069758.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  water['TasaTratamiento'].replace([float('inf'), float('-inf'), pd.NA], 0, inplace=True)


,NombreDepartamento,NombreMunicipio,Año,MuestrasEvaluadas,MuestrasTratadas,TasaTratamiento
0,Bolívar,El Guamo,2010,67,67,1.0
1,Bolívar,El Guamo,2010,67,67,1.0
2,Bolívar,El Guamo,2010,67,67,1.0
3,Bolívar,El Guamo,2010,67,67,1.0
4,Bolívar,El Guamo,2010,67,67,1.0


La TasaTratamiento representa la proporción de muestras de agua tratadas respecto al total de muestras evaluadas para cada registro. Esta tasa es un indicador directo de la cobertura y efectividad de las estrategias de tratamiento implementadas en un área geográfica específica durante un periodo determinado.

Cómo Interpretar la TasaTratamiento:

•	Valor de 1.0: Indica que el 100% de las muestras evaluadas fueron tratadas, lo que sugiere una cobertura completa y probablemente efectiva de los procedimientos de tratamiento de agua.

•	Valor menor que 1.0 pero mayor que 0: Sugiere que una parte de las muestras evaluadas fue tratada. Cuanto más cercano esté el valor a 1.0, mejor es la cobertura del tratamiento.

•	Valor de 0: Indica que ninguna de las muestras evaluadas fue tratada, lo que puede ser motivo de preocupación y podría requerir atención inmediata para entender las razones detrás de la falta de tratamiento.

##### Depuración y Selección de Variables

In [9]:
water = water.drop(['ResultadoMinimo', 'ResultadoMaximo', 'ResultadoPromedio'], axis=1)

Para obtener una visión más profunda de la metodología y justificación detrás de la eliminación de las columnas 'ResultadoMínimo', 'ResultadoMáximo' y 'ResultadoPromedio', les invitamos a consultar el archivo EDA_water_quality. Este documento contiene nuestro análisis exploratorio de datos, el cual destila criterios clave y revela insights significativos que han guiado la depuración de nuestro conjunto de datos. 

In [ ]:
columnas_a_eliminar = ['MuestrasTratadas', 'MuestrasEvaluadas', 'MuestrasSinTratar',
                      'NumeroParametrosMinimo', 'NumeroParametrosMaximo', 'EsAtipico']
water = water.drop(columns=columnas_a_eliminar)


water.head()

In [ ]:
new_column_names = [
    "año", 
    "nombre_departamento", 
    "div_dpto", 
    "nombre_municipio", 
    "divi_muni", 
    "irca_minimo", 
    "irca_maximo", 
    "irca_promedio", 
    "rango_irca", 
    "nombre_parametro_analisis", 
    "numero_parametros_promedio", 
    "TasaTratamiento"
    "TratamientoCategoría"   
]
water.columns = new_column_names
print(water.head())

### Dimensional Modeling